
<a name="top" id="top"></a>

  <div align="center">
    <h1>Diet Problem</h1>
    <a href="https://github.com/bernalde">David E. Bernal Neira</a>
    <br>
    <i>Davidson School of Chemical Engineering, Purdue University</i>
    <br>
    <i>Universities Space Research Association</i>
    <br>
    <i>NASA QuAIL</i>
    <br>
    <br>
   <a href="https://github.com/abdelane">Abigail B. Delaney</a>
    <br>
    <i>Davidson School of Chemical Engineering, Purdue University</i>
    <br>
    <i>Undergraduate Research Assistant</i>
    <br>
    <br>
    <a href="https://github.com/murraybj">Benjamin J. L. Murray</a>
    <br>
    <i>Davidson School of Chemical Engineering, Purdue University</i>
    <br>
    <i>Undergraduate Research Assistant</i>
    <br>
    <br>
    <a href="https://colab.research.google.com/github/abdelane/pyomogallery/blob/main/dietproblemAB/DietProblemAB.ipynb" target="_parent">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
    </a>
    <a href="https://secquoia.github.io/">
        <img src="https://img.shields.io/badge/🌲⚛️🌐-SECQUOIA-blue" alt="SECQUOIA"/>
    </a>
</div>

# The Diet Problem

## Summary

The goal of the Diet Problem is to select foods that satisfy daily nutritional requirements at minimum cost. This problem can be formulated as a linear program, for which constraints limit the number of calories and the amount of vitamins, minerals, fats, sodium, and cholesterol in the diet. Danzig (1990) notes that the diet problem was motivated by the US Army's desire to minimize the cost of feeding GIs in the field while still providing a healthy diet.

## Problem Statement

The Diet Problem can be formulated mathematically as a linear programming problem using the following model.  

### Sets

 $F$ = set of foods  
 $N$ = set of nutrients

### Parameters

 $c_i$ = cost per serving of food $i$, $\forall i \in F$  
 $a_{ij}$ = amount of nutrient $j$ in food $i$, $\forall i \in F, \forall j \in N$  
 $Nmin_j$ = minimum level of nutrient $j$, $\forall j \in N$  
 $Nmax_j$ = maximum level of nutrient $j$, $\forall j \in N$  
 $V_i$ = the volume per serving of food $i$, $\forall i \in F$  
 $Vmax$ = maximum volume of food consumed

### Variables
 $x_i$ = number of servings of food $i$ to consume

### Objective

Minimize the total cost of the food  
 $\min \sum_{i \in F} c_i x_i$

### Constraints

Limit nutrient consumption for each nutrient $j \in N$.  
 $Nmin_j \leq \sum_{i \in F} a_{ij} x_i \leq Nmax_j$, $\forall j \in N$

Limit the volume of food consumed  
 $\sum_{i \in F} V_i x_i \leq Vmax$

Consumption lower bound  
 $x_i \geq 0$, $\forall i \in F$

## Pyomo Formulation

We begin by importing the Pyomo package and creating a model object:

In [ ]:
# If using this on Google collab, we need to install the packages
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

# Let's start with Pyomo
if IN_COLAB:
    !pip install -q pyomo

#uploading other files from computer to colab
if IN_COLAB:
  !wget https://raw.githubusercontent.com/Pyomo/PyomoGallery/refs/heads/master/diet/diet.dat
  !wget https://raw.githubusercontent.com/Pyomo/PyomoGallery/refs/heads/master/diet/results.yml

--2024-09-24 01:45:17--  https://raw.githubusercontent.com/abdelane/pyomogallery/refs/heads/main/dietproblemAB/diet.dat?token=GHSAT0AAAAAACWWZI6C3DZSB5FWSF63MT6AZXSE4IQ
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1486 (1.5K) [text/plain]
Saving to: ‘diet.dat?token=GHSAT0AAAAAACWWZI6C3DZSB5FWSF63MT6AZXSE4IQ’

diet.dat?token=GHSA 100%[===================>]   1.45K  --.-KB/s    in 0s      

2024-09-24 01:45:18 (23.1 MB/s) - ‘diet.dat?token=GHSAT0AAAAAACWWZI6C3DZSB5FWSF63MT6AZXSE4IQ’ saved [1486/1486]

--2024-09-24 01:45:18--  https://raw.githubusercontent.com/abdelane/pyomogallery/refs/heads/main/dietproblemAB/results.yml?token=GHSAT0AAAAAACWWZI6DB4MT75UXRCNRFCAOZXSE52Q
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.

In [32]:
from pyomo.environ import *
#imports pyomo package
infinity = float('inf')
#sets infinity to a float value

model = AbstractModel()
#creates an abstract model named model

The sets $F$ and $N$ are declared abstractly using the `Set` component:

In [33]:
# Foods
model.F = Set()
#creates an empty set of foods that can be filled later

# Nutrients
model.N = Set()
#creates an empty set of nutrients that can be filled later

Similarly, the model parameters are defined abstractly using the `Param` component:

In [34]:
# Cost of each food
model.c    = Param(model.F, within=PositiveReals)
#creates a parameter for the cost of each food for every item in the set 
# of foods and all costs must be postive and real

# Amount of nutrient in each food
model.a    = Param(model.F, model.N, within=NonNegativeReals)
#creates a parameter for the amount of each nutrient in each food, so 
# it is a parameter for each combination of item from the set of foods and nutrients

# Lower and upper bound on each nutrient
model.Nmin = Param(model.N, within=NonNegativeReals, default=0.0)
    #parameter for the minimum amount of each nutrient, default is 0
model.Nmax = Param(model.N, within=NonNegativeReals, default=infinity)
    #parameter for the maximum amount of each nutrient, default is infinity

# Volume per serving of food
model.V    = Param(model.F, within=PositiveReals)
#parameter for the volume of each food within set F, must be positive and real

# Maximum volume of food consumed
model.Vmax = Param(within=PositiveReals)


The `within` option is used in these parameter declarations to define expected properties of the parameters.  This information is used to perform error checks on the data that is used to initialize the parameter components.

The `Var` component is used to define the decision variables:

In [35]:
# Number of servings consumed of each food
model.x = Var(model.F, within=NonNegativeIntegers)
    #integer variable for the number of servings of each food in set F, must be nonnegative

The `within` option is used to restrict the domain of the decision variables to the non-negative reals. This eliminates the need for explicit bound constraints for variables.

The `Objective` component is used to define the cost objective.  This component uses a rule function to construct the objective expression:

In [36]:
# Minimize the cost of food that is consumed
def cost_rule(model):
    #defines a function to minimize the cost of food in model
    return sum(model.c[i]*model.x[i] for i in model.F)
    #returns the sum of the cost of each food times the number of servings of that food i within the set of food F
model.cost = Objective(rule=cost_rule)
#returns the objective function for the model where the cost of food times number of servings in summed over all foods

Similarly, rule functions are used to define constraint expressions in the `Constraint` component:

In [37]:
# Limit nutrient consumption for each nutrient
def nutrient_rule(model, j):
    #defines constraint on the model using j to iterate over the set of nutrients and i to iterate over set of foods
    value = sum(model.a[i,j]*model.x[i] for i in model.F)
    #sums the amount of nutrient in each food times the number of servings for all foods in set F
    return inequality(model.Nmin[j], value, model.Nmax[j])
    #returns the inequality that the amount of nutrient in the food must be less than the max number of nutrients and more than the min number of nutrients
model.nutrient_limit = Constraint(model.N, rule=nutrient_rule)
#returns the constraint for the model that the nutrient limit must be satisfied for all nutrients

# Limit the volume of food consumed
def volume_rule(model):
    #defines another constraint on the volume of food consumed in the model
    return sum(model.V[i]*model.x[i] for i in model.F) <= model.Vmax
    #returns the constraint that the sum of the volume of food times the number of servings for each food in the set F and ensures it is less than the max volume
model.volume = Constraint(rule=volume_rule)
#returns the constraint for the model that the volume of food consumed must be less than the max volume

## Model Data

Since this is an abstract Pyomo model, the set and parameter values need to be provided to initialize the model.  The following data command file provides a synthetic data set:

In [39]:
!cat diet.dat

param:  F:                          c     V  :=
  "Cheeseburger"                 1.84   4.0  
  "Ham Sandwich"                 2.19   7.5  
  "Hamburger"                    1.84   3.5  
  "Fish Sandwich"                1.44   5.0  
  "Chicken Sandwich"             2.29   7.3  
  "Fries"                         .77   2.6  
  "Sausage Biscuit"              1.29   4.1  
  "Lowfat Milk"                   .60   8.0 
  "Orange Juice"                  .72  12.0 ;

param Vmax := 75.0;

param:  N:       Nmin   Nmax :=
        Cal      2000      .
        Carbo     350    375
        Protein    55      .
        VitA      100      .
        VitC      100      .
        Calc      100      .
        Iron      100      . ;

param a:
                               Cal  Carbo Protein   VitA   VitC  Calc  Iron :=
  "Cheeseburger"               510     34     28     15      6    30    20
  "Ham Sandwich"               370     35     24     15     10    20    20
  "Hamburger"                  500     42

Set data is defined with the `set` command, and parameter data is defined with the `param` command.

This data set considers the problem of designing a daily diet with only food from a fast food chain.

## Solution

Pyomo includes a `pyomo` command that automates the construction and optimization of models.  The GLPK solver can be used in this simple example:

Install the MIP solver GLPK and then solve the model.

In [ ]:
# Let's install the LP/MIP solver GLPK
if IN_COLAB:
    !apt-get install -y -qq glpk-utils

# Define the solver GLPK
if IN_COLAB:
    opt_glpk = SolverFactory('glpk', executable='/usr/bin/glpsol')
else:
    opt_glpk = SolverFactory('glpk')
# Here we could use another solver, e.g. gurobi or cplex
# opt_gurobi = SolverFactory('gurobi')

#creating a model instance for the abstract model using the data in diet.dat
instance = model.create_instance('diet.dat')

# We obtain the solution of model instance with GLPK
result_obj = opt_glpk.solve(instance, tee=False)
instance.display()

Model unknown

  Variables:
    x : Size=9, Index=F
        Key              : Lower : Value : Upper : Fixed : Stale : Domain
            Cheeseburger :     0 :   4.0 :  None : False : False : NonNegativeIntegers
        Chicken Sandwich :     0 :   0.0 :  None : False : False : NonNegativeIntegers
           Fish Sandwich :     0 :   1.0 :  None : False : False : NonNegativeIntegers
                   Fries :     0 :   5.0 :  None : False : False : NonNegativeIntegers
            Ham Sandwich :     0 :   0.0 :  None : False : False : NonNegativeIntegers
               Hamburger :     0 :   0.0 :  None : False : False : NonNegativeIntegers
             Lowfat Milk :     0 :   4.0 :  None : False : False : NonNegativeIntegers
            Orange Juice :     0 :   0.0 :  None : False : False : NonNegativeIntegers
         Sausage Biscuit :     0 :   0.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    cost : Size=1, Index=None, Active=True
        Key  : Active : Value
   

By default, the optimization results are stored in the file `results.yml`:

In [41]:
!cat results.yml

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 15.05
  Upper bound: 15.05
  Number of objectives: 1
  Number of constraints: 10
  Number of variables: 10
  Number of nonzeros: 77
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 89
      Number of created subproblems: 89
  Error rc: 0
  Time: 0.00977396965027
# ----------------------------------------------------------
#   Solution Information
# -----------------------------------

This solution shows that for about $15 per day, a person can get by with 4
cheeseburgers, 5 fries, 1 fish sandwich and 4 milks.

## References

* G.B. Dantzig. The Diet Problem, Interfaces 20(4), 1990, 43-47